In [2]:
import json, os
import datetime
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
# Helper function
def parse_json(infile):
    with open(infile) as f:
        data = json.load(f)
    df = json_normalize(json_normalize(data).loc[0,'messages'])
    df['date_time'] = [datetime.datetime.fromtimestamp(t/1000.0) for t in df['timestamp_ms']]
    df['date_time'] = pd.to_datetime(df['date_time'])
    df['year'] = [t.year for t in df['date_time']]
    df['month'] = [t.month for t in df['date_time']]
    df['day'] = [t.day for t in df['date_time']]
    df['hour'] = [t.hour for t in df['date_time']]
    df['weekday'] = [t.weekday() for t in df['date_time']]
    return df[['content', 'sender_name', 'timestamp_ms', 'type', 'date_time', 'year', 'month', 'day', 'hour', 'weekday']]

In [4]:
dataset_p = '../../data'
messages_p = dataset_p + '/messages/inbox/'

In [5]:
all_json = [messages_p + s + '/message.json' for s in sorted(os.listdir(messages_p))[1:]]
print("Total number of chats:", len(all_json))

Total number of chats: 339


In [ ]:
for j in range(len(all_json)):
    if j == 0:
        try:
            df = parse_json(all_json[j])
        except:
            continue
    else:
        try:
            df = pd.concat([df, parse_json(all_json[j])], 0)
        except:
            continue
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv(dataset_p + '/clean_messages.csv', index = None)